In [0]:
# provider-notebooks/Module 2 - Delta Sharing Deep Dive/Includes/Lab-Setup-2.ipynb

In [0]:
%run ./_common

In [0]:
# set variables in python and sql
config_values = [
    ("catalog", "techware_labs"),
    ("schema", "inventory"),
    ("internal_share_name", "warehouse_inventory"),
    ("internal_recipient_name", "regional_warehouse"),
]
DA = create_da_objects(config_values)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS IDENTIFIER(DA.catalog)
COMMENT 'TechWare Labs catalog for inventory sharing';
USE CATALOG IDENTIFIER(DA.catalog);

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS IDENTIFIER(DA.schema)
COMMENT 'Schema containing product and warehouse inventory data';
USE SCHEMA IDENTIFIER(DA.schema);

In [0]:
# Cleanup existing shares and recipients before setup
cleanup_commands = [
    "DROP SHARE IF EXISTS warehouse_inventory",
    "DROP RECIPIENT IF EXISTS regional_warehouse",
]

for cmd in cleanup_commands:
    try:
        spark.sql(cmd)
    except Exception as e:
        print(f"Note: {cmd} - {str(e)}")

In [0]:
%sql
-- Drop table if exists (to keep history clean)
DROP TABLE IF EXISTS products;

-- Create products table
CREATE TABLE products (
  product_id INT,
  product_name STRING,
  category STRING,
  unit_price DECIMAL(10,2),
  supplier_id INT,
  supplier_name STRING,
  reorder_level INT
)
COMMENT 'Product master data for inventory management';

-- Insert sample data
INSERT INTO products VALUES
  (1001, 'Wireless Mouse', 'Peripherals', 29.99, 501, 'TechSupply Co', 50),
  (1002, 'USB-C Hub', 'Accessories', 49.99, 502, 'ConnectPro', 30),
  (1003, 'Laptop Stand', 'Accessories', 79.99, 501, 'TechSupply Co', 25),
  (1004, 'Mechanical Keyboard', 'Peripherals', 149.99, 503, 'KeyMasters Inc', 20),
  (1005, 'Webcam HD', 'Peripherals', 89.99, 504, 'Vision Tech', 40),
  (1006, 'Monitor 27inch', 'Displays', 299.99, 505, 'Display Solutions', 15),
  (1007, 'Desk Lamp LED', 'Office', 39.99, 506, 'LightWorks', 60),
  (1008, 'Ergonomic Chair', 'Furniture', 399.99, 507, 'Comfort Designs', 10),
  (1009, 'Headset Pro', 'Peripherals', 129.99, 504, 'Vision Tech', 35),
  (1010, 'Docking Station', 'Accessories', 199.99, 502, 'ConnectPro', 20);

In [0]:
%sql
-- Drop table if exists (to keep history clean)
DROP TABLE IF EXISTS warehouse_stock;

-- Create warehouse stock table
CREATE TABLE warehouse_stock (
  stock_id INT,
  product_id INT,
  warehouse_location STRING,
  quantity_on_hand INT,
  last_restock_date DATE,
  warehouse_zone STRING
)
COMMENT 'Current warehouse inventory levels';

-- Insert sample warehouse stock data
INSERT INTO warehouse_stock VALUES
  (5001, 1001, 'North Warehouse', 245, '2024-09-15', 'Zone A'),
  (5002, 1002, 'North Warehouse', 156, '2024-09-20', 'Zone A'),
  (5003, 1003, 'East Warehouse', 89, '2024-09-10', 'Zone B'),
  (5004, 1004, 'West Warehouse', 67, '2024-09-25', 'Zone C'),
  (5005, 1005, 'North Warehouse', 198, '2024-09-18', 'Zone A'),
  (5006, 1006, 'East Warehouse', 45, '2024-09-12', 'Zone B'),
  (5007, 1007, 'West Warehouse', 312, '2024-09-22', 'Zone A'),
  (5008, 1008, 'South Warehouse', 28, '2024-09-05', 'Zone D'),
  (5009, 1009, 'North Warehouse', 134, '2024-09-17', 'Zone C'),
  (5010, 1010, 'East Warehouse', 76, '2024-09-14', 'Zone B'),
  (5011, 1001, 'East Warehouse', 189, '2024-09-16', 'Zone A'),
  (5012, 1002, 'West Warehouse', 92, '2024-09-21', 'Zone A'),
  (5013, 1004, 'South Warehouse', 51, '2024-09-08', 'Zone C'),
  (5014, 1006, 'West Warehouse', 33, '2024-09-11', 'Zone B'),
  (5015, 1008, 'North Warehouse', 19, '2024-09-06', 'Zone D');

In [0]:
display_config_values(config_values)